Load PUDL Data as example of Data Ingestion pipeline
Copyright (C) 2021 OS-Climate

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [ ]:
# pip install boto3
# pip install python-dotenv
# pip install trino
# pip install pandas
# pip install pyarrow

In [ ]:
from dotenv import dotenv_values, load_dotenv
from osc_ingest_trino import *
import os
import pathlib
import boto3
import pandas as pd
import trino
import io

Load Environment Variables

In [ ]:
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

Create an S3 resource for the bucket holding source data, then retrieve the relevant bucket and list objects

In [ ]:
s3_resource = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)

In [ ]:
bucket = s3_resource.Bucket(os.environ['S3_LANDING_BUCKET'])

# list all objects
#for obj in bucket.objects.all():
#    print(obj.key, obj.last_modified)
    
files = []
for file in bucket.objects.filter(Prefix='PUDL/pudl-v0.5.0/pudl_data/parquet/'):
    if file.key.endswith('parquet'):
        files.append(file.key)
files[0:10]

In [ ]:
from os.path import exists

for sqlite_name in ['pudl', 'ferc1']:
    sqlite_filepath = f'/opt/app-root/share/pudl/sqlite/{sqlite_name}.sqlite'
    if not exists(sqlite_filepath):
        os.makedirs('/opt/app-root/share/pudl/sqlite', mode=0o775, exist_ok=True)
        sqlite_src = s3_resource.Object(os.environ['S3_LANDING_BUCKET'],f'PUDL/pudl-v0.5.0/pudl_data/sqlite/{sqlite_name}.sqlite')
        sqlite_src.download_file(sqlite_filepath)
    else:
        print(f'{sqlite_filepath} exists')

Read the first parquet file into a Pandas data frame and view the results

In [ ]:
buffer = io.BytesIO()
parquet_file = s3_resource.Object(os.environ['S3_LANDING_BUCKET'],files[0])
parquet_file.download_fileobj(buffer)
df = pd.read_parquet(buffer)
df

In [ ]:
df.info(verbose=True)

Ingest the data onto the S3 Development Bucket

In [ ]:
tablename = 'Y95_AL'
df.to_parquet('/tmp/{tname}.parquet'.format(tname=tablename), index=False)

In [ ]:
# Create an S3 client
s3 = boto3.client(
    service_name="s3",
    endpoint_url=os.environ['S3_DEV_ENDPOINT'],
    aws_access_key_id=os.environ['S3_DEV_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_DEV_SECRET_KEY'],
)

In [ ]:
s3.upload_file(
    Bucket=os.environ['S3_DEV_BUCKET'],
    Key='trino/pudl/{tname}/{tname}.parquet'.format(tname=tablename),
    Filename='/tmp/{tname}.parquet'.format(tname=tablename)
)

In [ ]:
conn = trino.dbapi.connect(
    host=os.environ['TRINO_HOST'],
    port=int(os.environ['TRINO_PORT']),
    user=os.environ['TRINO_USER'],
    http_scheme='https',
    auth=trino.auth.JWTAuthentication(os.environ['TRINO_PASSWD']),
    verify=True,
)
cur = conn.cursor()

In [ ]:
# Show available schemas to ensure trino connection is set correctly
cur.execute('show schemas in osc_datacommons_dev')
cur.fetchall()

Create ingestion schema based on source data name and remove old tables if necessary

In [ ]:
cur.execute('create schema if not exists osc_datacommons_dev.pudl')
cur.fetchall()

In [ ]:
cur.execute('drop table if exists osc_datacommons_dev.pudl.' + tablename)
cur.fetchall()

In [ ]:
schema = create_table_schema_pairs(df)

tabledef = """create table if not exists osc_datacommons_dev.pudl.{tname}(
{schema}
) with (
    format = 'parquet',
    external_location = 's3a://{bucket}/trino/pudl/{tname}/'
)""".format(schema=schema,bucket=os.environ['S3_DEV_BUCKET'],tname=tablename)
print(tabledef)

# tables created externally may not show up immediately in cloud-beaver
cur.execute(tabledef)
cur.fetchall()

In [ ]:
cur.execute('SELECT * FROM osc_datacommons_dev.pudl.y95_al limit 10')
cur.fetchall()